In [ ]:
import zipfile
import os
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Dataset Loading

In [ ]:
zip_file_path = '/content/drive/MyDrive/MLMA Project/GROUPBY SPLIT/Use This Final Final (22 04 25)/augumented_final.zip' # Path to zipped dataset
extract_folder = '/content/Finaldata' # Path to unzip
if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print(f"Successfully extracted {zip_file_path} to {extract_folder}")
except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully extracted /content/drive/MyDrive/MLMA Project/GROUPBY SPLIT/Use This Final Final (22 04 25)/augumented_final.zip to /content/Finaldata


In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (299, 299)
EPOCHS = 10
train_dir = r"/content/Finaldata/augumented_final/train"
valid_dir = r"/content/Finaldata/augumented_final/valid"
test_dir  = r"/content/Finaldata/augumented_final/test"

In [ ]:
# Load train, validation and test sets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

Found 8958 files belonging to 8 classes.
Found 1282 files belonging to 8 classes.
Found 631 files belonging to 8 classes.
Classes: ['A', 'C', 'D', 'G', 'H', 'M', 'N', 'O']


In [ ]:
class_counts = np.zeros(num_classes, dtype=int)
for images, labels in train_ds.unbatch():
    class_counts[labels.numpy()] += 1
print("Class counts:", class_counts)
class_weights = {i: float(num_classes/count) for i, count in enumerate(class_counts)}   # Compute class weights
print("Class weights:", class_weights)

Class counts: [ 370  414 2262  388  182  338 4032  972]
Class weights: {0: 0.021621621621621623, 1: 0.01932367149758454, 2: 0.0035366931918656055, 3: 0.020618556701030927, 4: 0.04395604395604396, 5: 0.023668639053254437, 6: 0.001984126984126984, 7: 0.00823045267489712}


XceptionNet Training and Evaluation

In [ ]:
# Function to build XceptionNet model

def build_xception_model(learning_rate, weight_decay):
    base_model = tf.keras.applications.Xception(
        weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,)
    )
    base_model.trainable = False  # Freeze the base model
    inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = tf.keras.applications.xception.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax",
                           kernel_regularizer=regularizers.l2(weight_decay))(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Define hyperparameters
learning_rates = [1e-3, 5e-4]
weight_decays = [0.0, 1e-4]

best_val_acc = 0.0
best_config = None
best_model = None

# Hyperparamter tuning
for lr in learning_rates:
    for wd in weight_decays:
        print(f"\nTraining configuration: lr={lr}, weight_decay={wd}")
        model = build_xception_model(learning_rate=lr, weight_decay=wd)
        earlystop = callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True) # Early stopping

        # Train the model
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=EPOCHS,
            callbacks=[earlystop],
            class_weight=class_weights,
            verbose=1
        )

        max_val_acc = max(history.history['val_accuracy'])
        print(f"Config: lr={lr}, wd={wd} -- Best validation accuracy: {max_val_acc:.4f}")
        if max_val_acc > best_val_acc:
            best_val_acc = max_val_acc
            best_config = (lr, wd)
            best_model = model   # Model with the best validation accuracy

print(f"\nBest hyperparameters: lr={best_config[0]}, weight_decay={best_config[1]}")
print(f"Best validation accuracy: {best_val_acc:.4f}")


Training configuration: lr=0.001, weight_decay=0.0
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 43s 117ms/step - accuracy: 0.2238 - loss: 0.0126 - val_accuracy: 0.3284 - val_loss: 1.6914
Epoch 2/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.3491 - loss: 0.0095 - val_accuracy: 0.2387 - val_loss: 1.8703
Epoch 3/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.3743 - loss: 0.0088 - val_accuracy: 0.3175 - val_loss: 1.7728
Epoch 4/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.4004 - loss: 0.0083 - val_accuracy: 0.2800 - val_loss: 1.7894
Config: lr=0.001, wd=0.0 -- Best validation accuracy: 0.3284

Training configuration: lr=0.001, weight_decay=0.0001
Epoch 1/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 22s 56ms/step - accuracy: 0.2341 - loss: 0.0136 - val_accuracy: 0.2379 - val_loss: 1.8563
Epoch 2/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - accuracy: 0.3033 - loss: 0.0109 - val_accuracy: 0.3237 - val_loss: 1.7062
Epoc

In [ ]:
# Evaluate the best model on the test set

y_true = []
y_pred = []
for images, labels in test_ds:
    preds = best_model.predict(images)
    y_true.extend(labels.numpy()) # True labels
    y_pred.extend(np.argmax(preds, axis=1).tolist()) # Predictions
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute metrics
acc       = accuracy_score(y_true, y_pred)
prec_w    = precision_score(y_true, y_pred, average='weighted', zero_division=0)
rec_w     = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1_w      = f1_score(y_true, y_pred, average='weighted', zero_division=0)

# Print results
print(f"Overall Test Accuracy : {acc:.4f}")
print(f"Weighted Precision   : {prec_w:.4f}")
print(f"Weighted Recall      : {rec_w:.4f}")
print(f"Weighted F1 Score    : {f1_w:.4f}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step
Overall Test Accuracy : 0.3978
Weighted Precision   : 0.4869
Weighted Recall      : 0.3978
Weighted F1 Score    : 0.4194



In [ ]:
model_save_path = "/content/best_model.keras"
best_model.save(model_save_path)   # Save the best model
print(f"Best model saved to: {model_save_path}")

Best model saved to: /content/best_model.keras


MobileNetV3 Training and Evaluation

In [ ]:
# Function to build MobileNetV3 model

def build_mobilenetv3_model(lr, wd):
    base = tf.keras.applications.MobileNetV3Large(
        input_shape=IMG_SIZE + (3,),
        include_top=False, weights="imagenet"
    )
    base.trainable = False # Freeze the base model
    inputs = layers.Input(shape=IMG_SIZE + (3,))
    x = tf.keras.applications.mobilenet_v3.preprocess_input(inputs)
    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(
        num_classes, activation="softmax",
        kernel_regularizer=regularizers.l2(wd)
    )(x)
    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [ ]:
# Define hyperparameters
learning_rates = [1e-3, 5e-4]
weight_decays  = [0.0, 1e-4]

best_val_acc = 0.0
best_config  = None
best_model   = None.

# Hyperparameter tuning
for lr in learning_rates:
    for wd in weight_decays:
        print(f"\nTraining MobileNetV3 with lr={lr}, weight_decay={wd}")
        model = build_mobilenetv3_model(lr, wd)
        es = callbacks.EarlyStopping(
            monitor="val_accuracy", patience=3,
            restore_best_weights=True
        )                                             # Early stopping

        # Train the model
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=EPOCHS,
            class_weight=class_weights,
            callbacks=[es],
            verbose=1
        )
        val_acc = max(history.history["val_accuracy"])
        print(f" → Best val_accuracy: {val_acc:.4f}")
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_config  = (lr, wd)
            best_model   = model  # Model wiht the best validation accuracy

print(f"\nBest config: lr={best_config[0]}, weight_decay={best_config[1]}")
print(f"Best validation accuracy: {best_val_acc:.4f}")


Training MobileNetV3 with lr=0.001, weight_decay=0.0


/usr/local/lib/python3.11/dist-packages/keras/src/applications/mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


Epoch 1/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.2355 - loss: 0.0130 - val_accuracy: 0.4080 - val_loss: 1.5604
Epoch 2/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.3926 - loss: 0.0091 - val_accuracy: 0.3557 - val_loss: 1.6333
Epoch 3/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4090 - loss: 0.0084 - val_accuracy: 0.3588 - val_loss: 1.6661
Epoch 4/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4528 - loss: 0.0077 - val_accuracy: 0.4165 - val_loss: 1.4676
Epoch 5/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.4732 - loss: 0.0075 - val_accuracy: 0.4095 - val_loss: 1.5350
Epoch 6/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4820 - loss: 0.0071 - val_accuracy: 0.4267 - val_loss: 1.4634
Epoch 7/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4982 - loss: 0.0069 - val_accuracy: 0.4493 - val_loss: 1.4748
Epoch 8/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.5037 - loss: 0.0067 - val_acc

In [ ]:
# Evaluate the best model on the test set
y_true = []
y_pred = []
for images, labels in test_ds:
    preds = best_model.predict(images)
    y_true.extend(labels.numpy()) # True labels
    y_pred.extend(np.argmax(preds, axis=1).tolist()) # Predictions
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute metrics
acc       = accuracy_score(y_true, y_pred)
prec_w    = precision_score(y_true, y_pred, average='weighted', zero_division=0)
rec_w     = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1_w      = f1_score(y_true, y_pred, average='weighted', zero_division=0)

# Print results
print("\nMobileNetV3 Test Results:")
print(f"Overall Test Accuracy : {acc:.4f}")
print(f"Weighted Precision   : {prec_w:.4f}")
print(f"Weighted Recall      : {rec_w:.4f}")
print(f"Weighted F1 Score    : {f1_w:.4f}\n")
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

model_save_path = "/content/best_mobilenetv3_model.keras"
best_model.save(model_save_path) # Save best model
print(f"Best MobileNetV3 model saved to: {model_save_path}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

MobileNetV3 Test Results:
Overall Test Accuracy : 0.4976
Weighted Precision   : 0.5101
Weighted Recall      : 0.4976
Weighted F1 Score    : 0.4906

              precision    recall  f1-score   support

           A       0.33      0.38      0